# Establishing Pipelines

Now that I have the structure of the transformations that I am going to apply to the data, I will create pipelines to facilitate the processes

## Data separation

In [ ]:
import pandas as pd
import numpy as np

# Read the data
route_employee_survey = '/content/drive/MyDrive/workplace-engineering-take-home-test-main/src/employee_survey_data.csv'
route_general_data = '/content/drive/MyDrive/workplace-engineering-take-home-test-main/src/general_data.csv'
route_manager_survey = '/content/drive/MyDrive/workplace-engineering-take-home-test-main/src/manager_survey_data.csv'

# Transform data into DataFrame
employee_survey = pd.read_csv(route_employee_survey, index_col="EmployeeID")
general_data = pd.read_csv(route_general_data, index_col="EmployeeID")
manager_survey = pd.read_csv(route_manager_survey, index_col="EmployeeID")

# Merge regular data sources into one dataframe
data = employee_survey.merge(general_data, on='EmployeeID')
data = data.merge(manager_survey, on='EmployeeID')

# Drop rows with null values of attrition
data.Regular = data.Regular.dropna(subset=['Attrition'])

# Separate target from predictors
y = data.Regular.Attrition
X = data.drop([('Regular', 'Attrition')], axis=1)

# Select numerical columns
columns_numerical = X.select_dtypes(include=[np.number]).columns

# Select categorical columns
columns_categorical = X.select_dtypes(include=["object"]).columns

# For Ordinal Encoding
columns_ordinal = ["BusinessTravel", "MaritalStatus", "Gender", "Over18"]

#For OneHot Encoding
columns_one_hot = columns_categorical.drop(columns_ordinal)

# Select numerical columns except TotalWorkingYears
data_int_cols = columns_numerical.drop("TotalWorkingYears")

## Defining Preprocesing steps

### Numerical imputation

Creating a custom transformer for using with TotalWorkingYears column

In [ ]:
from sklearn.impute import SimpleImputer

class WorkingYearsTransformer:
    def __init__(self):
        self.gen_imputer = SimpleImputer(strategy='median')

    def fit(self, X, Y=None):
        self.gen_imputer.fit(X[data_int_cols])

        return self

    def transform(self, df):
        transformed_df = df.copy()

        transformed_df[data_int_cols] = self.gen_imputer.transform(df[data_int_cols])
        transformed_df.loc[transformed_df.TotalWorkingYears.isna(), "TotalWorkingYears"] = transformed_df.loc[transformed_df.TotalWorkingYears.isna(), "YearsAtCompany"]

        return transformed_df.to_numpy()

numerical_transformer = WorkingYearsTransformer()

### Categorical imputation and encoding

In [ ]:
# Imputer for categorical columns

cat_imputer = SimpleImputer(strategy='most_frequent')

Preprocessing for categorical Ordinal data

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Encoder for marital status, travel frequency and over 18.
ordinal_encoder_travel = OrdinalEncoder(
    categories=[np.array(['Non-Travel', 'Travel_Rarely', 'Travel_Frequently'])],
    handle_unknown='use_encoded_value',
    unknown_value=-1
    )
ordinal_encoder_marital = OrdinalEncoder(
    categories=[np.array(['Single', 'Married', 'Divorced'])],
    handle_unknown='use_encoded_value',
    unknown_value=-1
    )
ordinal_encoder_18 = OrdinalEncoder(
    categories=[np.array(['N', 'Y'])],
    handle_unknown='use_encoded_value',
    unknown_value=-1
    )

A custom transformer for ordinal categorical data to distinguish between the columns BusinessTravel, MaritalStatus and Over18

In [ ]:
class Categorical_Ordinal_Transformer:
    def __init__(self):
        self.temp_imputer = SimpleImputer(strategy='most_frequent')
        self.ordinal_encoder_travel = ordinal_encoder_travel
        self.ordinal_encoder_marital = ordinal_encoder_marital
        self.ordinal_encoder_gender = OrdinalEncoder()
        self.ordinal_encoder_18 = ordinal_encoder_18

    def fit(self, X, Y=None):
        imputed_X = X[columns_ordinal].copy()

        self.temp_imputer.fit(imputed_X)

        imputed_X.iloc[:, :] = self.temp_imputer.transform(imputed_X)

        self.ordinal_encoder_travel.fit(imputed_X[["BusinessTravel"]])
        self.ordinal_encoder_marital.fit(imputed_X[["MaritalStatus"]])
        self.ordinal_encoder_gender.fit(imputed_X[["Gender"]])
        self.ordinal_encoder_18.fit(imputed_X[["Over18"]])

        return self

    def transform(self, df):
        transformed_df = df[columns_ordinal].copy()

        transformed_df.iloc[:, :] = self.temp_imputer.transform(transformed_df)

        transformed_df.loc[:, "BusinessTravel"] = self.ordinal_encoder_travel.transform(transformed_df[["BusinessTravel"]])
        transformed_df.loc[:, "MaritalStatus"] = self.ordinal_encoder_marital.transform(transformed_df[["MaritalStatus"]])
        transformed_df.loc[:, "Gender"] = self.ordinal_encoder_gender.transform(transformed_df[["Gender"]])
        transformed_df.loc[:, "Over18"] = self.ordinal_encoder_18.transform(transformed_df[["Over18"]])

        return transformed_df


categorical_ordinal_transformer = Categorical_Ordinal_Transformer()

Preprocessing for categorical not Ordinal data

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

onehot_encoder = OneHotEncoder(
    handle_unknown='infrequent_if_exist',
    sparse_output=False,
    )

categorical_oneHot_transformer = Pipeline(steps=[
    ('imputer', cat_imputer),
    ('oneHot', onehot_encoder)
])

Ordinal and OneHot separation

Needed a custom transformer here to manage the loss of the columns for the inner transformer

In [ ]:
class Preprocesor_Transformer:
    def __init__(self):
        self.categorical_ordinal_transformer = categorical_ordinal_transformer
        self.categorical_oneHot_transformer = categorical_oneHot_transformer

    def fit(self, X, Y=None):
        ordinal_df = X[columns_ordinal]
        oneHot_df = X[columns_one_hot]

        self.categorical_ordinal_transformer = self.categorical_ordinal_transformer.fit(ordinal_df)
        self.categorical_oneHot_transformer = self.categorical_oneHot_transformer.fit(oneHot_df)

        return self

    def transform(self, df):
        transformed_df = df.copy()

        transformed_df.loc[:, columns_ordinal] = self.categorical_ordinal_transformer.transform(transformed_df[columns_ordinal])

        hot_encoded_data = self.categorical_oneHot_transformer.transform(transformed_df[columns_one_hot])
        hot_encoded_columns = categorical_oneHot_transformer.get_feature_names_out()

        hot_encoded_df = pd.DataFrame(hot_encoded_data, columns=hot_encoded_columns, index=transformed_df.index)
        transformed_df = pd.concat([transformed_df[columns_ordinal], hot_encoded_df], axis=1)

        return transformed_df


categorical_transformer = Preprocesor_Transformer()

### Categorical and Numerical separation

Same as before

In [ ]:
class Cat_Num_Separator:
    def __init__(self):
        self.numerical_transformer = numerical_transformer
        self.categorical_transformer = categorical_transformer

    def fit(self, X, Y=None):
        num_df = X[columns_numerical]
        cat_df = X[columns_categorical]

        self.numerical_transformer = self.numerical_transformer.fit(num_df)
        self.categorical_transformer = self.categorical_transformer.fit(cat_df)

        return self

    def transform(self, df):
        num_vals = self.numerical_transformer.transform(df[columns_numerical])
        cat_vals = self.categorical_transformer.transform(df[columns_categorical])

        return num_vals.merge(cat_vals, on="EmployeeID")


catNumSeparator = Cat_Num_Separator()